In [1]:
!pip install datasets
!pip install fais



In [2]:

import pandas as pd
from datasets import load_dataset, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import Trainer, TrainingArguments
from transformers import RagRetriever, RagSequenceForGeneration, RagTokenizer
import fais



In [3]:
#Crear el Dataset
data_extended = [
    # Preguntas y respuestas existentes
    {"instruction": "¿Cuál es el horario de apertura?", "response": "Abrimos de 18:00 a 02:00 de martes a domingo."},
    {"instruction": "¿Tienen opciones vegetarianas?", "response": "Sí, tenemos varias opciones vegetarianas, como nuestras croquetas de espinacas y nuestra ensalada de quinoa."},
    {"instruction": "¿Hacen reservas para grupos grandes?", "response": "Sí, aceptamos reservas para grupos grandes. Por favor, llámenos para hacer su reserva."},
    {"instruction": "¿Qué tipo de cervezas tienen?", "response": "Tenemos una variedad de cervezas locales e internacionales, incluyendo cervezas artesanales, rubias, tostadas y negras."},
    {"instruction": "¿Dónde se encuentra el bar?", "response": "Estamos ubicados en la calle mayor número 12."},
    {"instruction": "¿Tienen música en directo?", "response": "Sí, los viernes y los sábados tenemos música en directo a partir de las 22:00."},
    {"instruction": "¿Cuál es la especialidad de la casa?", "response": "Nuestra especialidad son los cócteles de autor y la hamburguesa gourmet con queso brie y cebolla caramelizada."},
    {"instruction": "¿Tienen opciones sin gluten?", "response": "Sí, tenemos algunas opciones sin gluten, como nuestra ensalada de pollo y las papas fritas."},
    {"instruction": "¿Hay menú del día?", "response": "Sí, ofrecemos un menú del día con opciones de primer y segundo plato a un precio fijo. Puede ver las opciones en nuestra pizarra."},
    {"instruction": "¿Puedo pagar con tarjeta?", "response": "Sí, aceptamos tarjetas de crédito y débito, así como pagos en efectivo."},
    {"instruction": "¿Hay opciones para veganos?", "response": "Sí, tenemos varias opciones veganas, como nuestra pizza de verduras y el hummus con pan de pita."},
    {"instruction": "¿Qué cócteles recomiendan?", "response": "Recomendamos el Mojito de fresa o el cóctel de la casa, que lleva una mezcla de frutas tropicales y ron."},
    {"instruction": "¿Tienen servicio a domicilio?", "response": "Actualmente no ofrecemos servicio a domicilio, pero puede recoger su pedido en el local."},
    {"instruction": "¿Hay menú para niños?", "response": "Sí, tenemos un menú especial para niños con opciones como pizza, pasta y jugos naturales."},
    {"instruction": "¿Cuál es el precio de un mojito?", "response": "El mojito cuesta 6 euros."},
    {"instruction": "¿Tienen bebidas sin alcohol?", "response": "Sí, ofrecemos una variedad de bebidas sin alcohol, como zumos naturales, cócteles sin alcohol y refrescos."},
    {"instruction": "¿Es necesario hacer reserva para una mesa?", "response": "No es necesario hacer reserva, pero si vienes en grupo grande o en hora punta, te recomendamos hacerlo."},
    {"instruction": "¿Hay aparcamiento cerca?", "response": "Sí, hay aparcamiento público a una calle de distancia del bar."},
    {"instruction": "¿Tienen Wi-Fi?", "response": "Sí, tenemos Wi-Fi gratuito para nuestros clientes. La clave está en la carta."},
    {"instruction": "¿Ofrecen descuentos para estudiantes?", "response": "Sí, ofrecemos un 10% de descuento en bebidas a los estudiantes con su carnet."},
    {"instruction": "¿Tienen cócteles sin alcohol?", "response": "Sí, tenemos varias opciones de cócteles sin alcohol como el cóctel de piña colada sin ron o el mojito de hierbabuena sin alcohol."},
    {"instruction": "¿Cuál es el precio de un gin-tonic?", "response": "El gin-tonic cuesta 8 euros."},
    {"instruction": "¿Puedo cambiar mi bebida si no me gusta?", "response": "Claro, si no estás satisfecho con tu bebida, avísanos y con gusto te la cambiamos."},
    {"instruction": "¿Hay algún tipo de oferta en cócteles durante la semana?", "response": "Sí, los miércoles tenemos 2x1 en cócteles seleccionados entre las 18:00 y las 20:00."},
    {"instruction": "¿Tienen cócteles sin alcohol para niños?", "response": "Sí, tenemos cócteles especiales para niños, como el cóctel de frutas tropicales sin alcohol."},
    {"instruction": "¿Puedo llevar mi perro al bar?", "response": "Sí, aceptamos mascotas en nuestra terraza exterior. Solo asegúrate de que esté con correa."},
    {"instruction": "¿A qué hora cierran los fines de semana?", "response": "Los fines de semana cerramos a las 03:00."},
    {"instruction": "¿Tienen opciones de postres?", "response": "Sí, ofrecemos varios postres como tarta de queso, brownies y helados artesanales."},
    {"instruction": "¿Puedo pedir una bebida sin alcohol si estoy embarazada?", "response": "Sí, tenemos varias opciones seguras y deliciosas para disfrutar sin alcohol."},
    {"instruction": "¿Tienen mojitos de frutas?", "response": "Sí, tenemos mojitos de frutas como mango, fresa y piña."},
    {"instruction": "¿Ofrecen catering o eventos privados?", "response": "Sí, organizamos eventos privados y ofrecemos catering para celebraciones, reuniones o fiestas."},
    {"instruction": "¿Tienen karaoke en el bar?", "response": "Sí, los jueves por la noche tenemos una noche de karaoke a partir de las 21:00."},
    {"instruction": "¿A qué hora empiezan las promociones en bebidas?", "response": "Las promociones en bebidas comienzan todos los días a las 18:00 hasta las 20:00."},
    {"instruction": "¿Tienen opciones sin azúcar?", "response": "Sí, tenemos opciones de bebidas sin azúcar, como refrescos dietéticos o cócteles sin azúcar añadida."},
    {"instruction": "¿Ofrecen clases de coctelería?", "response": "Sí, ofrecemos clases de coctelería los sábados por la tarde. Puedes reservar tu plaza llamándonos."},
    {"instruction": "¿Tienen cócteles sin alcohol para adultos?", "response": "Sí, tenemos cócteles como el 'Virgin Mojito' o 'Cero Piña Colada' para quienes prefieren bebidas sin alcohol."},
    {"instruction": "¿Tienen sillas altas para niños?", "response": "Sí, tenemos sillas altas para que los niños se sienten cómodos mientras disfrutan de su comida."},
    {"instruction": "¿Ofrecen bebidas sin lactosa?", "response": "Sí, ofrecemos bebidas sin lactosa, como café con leche de almendras o avena."},
    {"instruction": "¿Puedo hacer una fiesta en el bar?", "response": "Sí, organizamos fiestas privadas. Contáctanos para detalles y disponibilidad."},
    {"instruction": "¿Tienen opción de barra libre para eventos?", "response": "Sí, ofrecemos paquetes de barra libre para eventos. Pide más información al personal."},
    {"instruction": "¿Qué métodos de pago aceptan?", "response": "Aceptamos efectivo, tarjetas de crédito y débito, así como pagos móviles como Apple Pay y Google Wallet."},
    {"instruction": "¿Tienen opciones de comida para llevar?", "response": "Sí, todas nuestras opciones de menú están disponibles para llevar. Solo tienes que pedir en el mostrador."},
    {"instruction": "¿Qué medidas de seguridad tienen contra el COVID-19?", "response": "Seguimos estrictamente todas las medidas sanitarias recomendadas, incluyendo desinfección frecuente, uso de mascarillas y disponibilidad de desinfectante de manos."},
    {"instruction": "¿Organizan eventos temáticos?", "response": "Sí, organizamos eventos temáticos regularmente. Puedes seguirnos en nuestras redes sociales para estar al tanto de los próximos eventos."},
    {"instruction": "¿Tienen menú de temporada?", "response": "Sí, ofrecemos un menú de temporada que cambia cada tres meses para incluir ingredientes frescos y platos de temporada."},
    {"instruction": "¿Puedo reservar una mesa online?", "response": "Sí, puedes reservar una mesa a través de nuestra página web o mediante nuestras redes sociales."}
]

# Crear el DataFrame
df_extended = pd.DataFrame(data_extended)

import os

# Verificar si el directorio existe, si no, crearlo
directory = '/content/drive/MyDrive/IA'
if not os.path.exists(directory):
    os.makedirs(directory)

# Guardar el DataFrame como archivo CSV
file_path_extended = os.path.join(directory, 'bar_chatbot_dataset_extended.csv')
df_extended.to_csv(file_path_extended, index=False)

file_path_extended


'/content/drive/MyDrive/IA/bar_chatbot_dataset_extended.csv'

***Cargo y tokenizo el dataset utilizando la biblioteca datasets de Hugging Face ***

In [4]:
#Cargar y Tokenizar el Dataset
df_extended = pd.read_csv(file_path_extended)

#convertir el DataFrame a Dataset de Hugging Face
dataset_extended = Dataset.from_pandas(df_extended)


In [5]:
!pip install flash-attn


In [ ]:
#Cargar el modelo preentrenado
model = AutoModelForCausalLM.from_pretrained('llmware/dragon-deci-7b-v0')
#Cargar el tokenizador
tokenizer = AutoTokenizer.from_pretrained('llmware/dragon-deci-7b-v0')

dataset_extended = Dataset.from_pandas(df)
#Tokenizar las preguntas y respuestas
def tokenize_function(examples):
    return tokenizer(examples['instruction'], examples['response'], truncation=True, padding="max_length")

tokenized_dataset = dataset_extended.map(tokenize_function, batched=True)

# Dividir el dataset en train y eval
train_dataset = tokenized_dataset.train_test_split(test_size=0.1)['train']
eval_dataset = tokenized_dataset.train_test_split(test_size=0.1)['test']


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


The repository for llmware/dragon-deci-7b-v0 contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/llmware/dragon-deci-7b-v0.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y
The repository for llmware/dragon-deci-7b-v0 contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/llmware/dragon-deci-7b-v0.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


*** Ahora realizamos el Fine-Tuning del Modelo, para ello configuramos los argumentos de entrenamiento y realizamos el ajuste fino del modelo ***

In [2]:
#Definir los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

#Crear el modelo
trainer.model()

#Guardar el modelo afinado
model.save_pretrained('./fine_tuned_model')
tokenizer.save_pretrained('./fine_tuned_model')


NameError: name 'TrainingArguments' is not defined

*** Configuro el pipeline de RAG para integrar el modelo con capacidades de recuperación de información***

In [3]:
#Crear el Pipeline de RAG
#Cargar el pipeline de RAG
retriever = RagRetriever.from_pretrained("facebook/rag-token-base")
rag_model = RagSequenceForGeneration.from_pretrained("facebook/rag-token-base", retriever=retriever)
rag_tokenizer = RagTokenizer.from_pretrained("facebook/rag-token-base")

#Indexar las respuestas
response = df_extended['response'].tolist()
response_encoding = tokenizer(response, padding=True, truncation=True, return_tensors="pt")
dimension = response_encoding['input_ids'].shape[-1]
index = faiss.IndexFlatL2(dimension)
faiss.normalize_L2(response_encoding['input_ids'])
index.add(response_encoding['input_ids'].numpy())


NameError: name 'RagRetriever' is not defined

***Defino el prompt que utilizaremos para generar las respuestas**

In [ ]:
#Crear el Prompt Engineering
def create_prompt(question, context):
  prompt = f"Pregunta: {question}\nContexto: {context}\nRespuesta:"
  return prompt

***Implemneto la función de preguntas y respuestas utilizando el prompt y el modelo ajustado***

In [ ]:
#Generar Respuestas utilizando el Prompt y el Modelo Ajustado
def answer_question(question, model, tokenizer, index, responses):
  #Tokenizar la pregunta
  question_encoding = tokenizer(question, return_tensors="pt")

  #Buscar la respuesta más cercana en el índice
  D, I = index.search(question_encoding['input_ids'].numpy(), k=1)
  best_response = responses[I[0][0]]

  #Crear el prompt
  prompt = create_prompt(question, best_response)

  #Generar la respuesta usando el modelo de RAG
  inputs = tokenizer(prompt, return_tensors="pt")
  outputs = model.generate(**inputs)
  answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

  return answer




***Pruebo el sistema con tosas las preguntas en tu dataset para asegurarme de que funciona correctamente***

In [ ]:
questions = df_extended['instruction'].tolist()

for question in questions:
  response = answer_question(question, rag_model, rag_tokenizer, index, response)
  print(f"Pregunta: {question}\nRespuesta: {response}\n")